In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm import tqdm_notebook
import warnings
import multiprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.svm import NuSVC
from scipy.optimize import minimize  
warnings.filterwarnings('ignore')

In [2]:
train1 = pd.read_csv('train.csv')
test1 = pd.read_csv('test.csv')
cols = [c for c in train1.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

In [9]:
def instant_model(train, test, col, clf = [QuadraticDiscriminantAnalysis(0.5), 
                  KNeighborsClassifier(n_neighbors = 7, p = 2, weights = 'distance'), 
                  NuSVC(probability = True, kernel = 'poly')]):
    oof = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    for i in tqdm_notebook(range(512)):

        train2 = train[train['wheezy-copper-turtle-magic'] == i]
        test2 = test[test['wheezy-copper-turtle-magic'] == i]
        idx1 = train2.index
        idx2 = test2.index

        data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
        # feature selection
        data2 = VarianceThreshold(threshold = 2).fit_transform(data[cols])
        
        train3 = pd.DataFrame(data2[:train2.shape[0]], index = idx1)
        train3['target'] = train2['target']
        test3 = pd.DataFrame(data2[train2.shape[0]:], index = idx2)
        
        
        
        # Kfold
        skf = StratifiedKFold(n_splits=11, random_state=42)
        for train_index, test_index in skf.split(train3, train3['target']):
            # clf
            clf_qda = clf[0]
            clf_knn = clf[1]
            clf_svm = clf[2]
            
            X_train = train3.iloc[train_index, :].drop(["target"], axis = 1)
            X_test = train3.iloc[test_index, :].drop(["target"], axis = 1)
            y_train = train3.iloc[train_index, :]['target']
            y_test = train3.iloc[test_index, :]['target']
            
            clf_qda.fit(X_train, y_train)
            clf_knn.fit(X_train, y_train)
            clf_svm.fit(X_train, y_train)            

            # output
#             train_prob_qda = clf_qda.predict_proba(X_train)[:,1]
#             train_prob_knn = clf_knn.predict_proba(X_train)[:,1]
#             train_prob_svm = clf_svm.predict_proba(X_train)[:,1]            
            
            test_prob_qda = clf_qda.predict_proba(X_test)[:,1]
            test_prob_knn = clf_knn.predict_proba(X_test)[:,1]
            test_prob_svm = clf_svm.predict_proba(X_test)[:,1]
            
            pred_dict = {0: roc_auc_score(y_test, test_prob_qda), 
                         1: roc_auc_score(y_test, test_prob_knn), 
                         2: roc_auc_score(y_test, test_prob_svm)}
                        
            pred_test = [test_prob_qda, test_prob_knn, test_prob_svm]            
                        
            oof[idx1[test_index]] = pred_test[max(pred_dict, key=pred_dict.get)]
            
            preds_list = [clf_qda.predict_proba(test3)[:,1]/ skf.n_splits,
                          clf_knn.predict_proba(test3)[:,1]/ skf.n_splits,
                          clf_svm.predict_proba(test3)[:,1]/ skf.n_splits,]
            # bagging
            preds[idx2] += preds_list[max(pred_dict, key = pred_dict.get)]
            names = ['QDA', 'KNN', 'SVM']
            print("Chunk {0} Fold {1} and method {2}".format(i, roc_auc_score(y_test, pred_test[max(pred_dict, key = pred_dict.get)]),names[max(pred_dict, key = pred_dict.get)]))

    auc = roc_auc_score(train['target'], oof)
    print(f'AUC: {auc:.5}')
    
    return oof, preds

In [4]:
def get_newtrain(train, test, preds, oof):
    # get useful train set from train and test data
    # get useful test 
    test['target'] = preds
    test.loc[test['target'] > 0.99, 'target'] = 1
    test.loc[test['target'] < 0.01, 'target'] = 0
    usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]

    # get useful train 
    new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
    new_train.loc[oof > 0.99, 'target'] = 1
    new_train.loc[oof < 0.01, 'target'] = 0
    return new_train

In [10]:
oof_temp, preds_temp = instant_model(train1, test1, cols)
#oof_temp, preds_temp = instant_model(train1, test1, cols, clf = DecisionTreeClassifier)

#oof_temp1, preds_temp1 = instant_model(train, test, cols, clf = NuSVC(probability = True, kernel = 'poly'))
#oof_temp2, preds_temp2 = instant_model(train1, test1, cols, clf = KNeighborsClassifier(n_neighbors = 7, p = 2, weights = 'distance'))


Chunk 0 Fold 0.9233333333333332 and method KNN
Chunk 0 Fold 0.9208333333333334 and method KNN
Chunk 0 Fold 0.9633333333333334 and method QDA
Chunk 0 Fold 0.9700000000000001 and method QDA
Chunk 0 Fold 0.9983333333333333 and method QDA
Chunk 0 Fold 1.0 and method QDA
Chunk 0 Fold 0.9600000000000001 and method QDA
Chunk 0 Fold 0.96875 and method KNN
Chunk 0 Fold 0.8975694444444444 and method QDA
Chunk 0 Fold 0.9600694444444444 and method QDA
Chunk 0 Fold 0.9927536231884058 and method QDA
Chunk 1 Fold 0.9202898550724637 and method QDA
Chunk 1 Fold 1.0 and method QDA
Chunk 1 Fold 0.9728260869565217 and method QDA
Chunk 1 Fold 0.9547101449275363 and method QDA
Chunk 1 Fold 0.9734848484848485 and method KNN
Chunk 1 Fold 0.9943181818181819 and method QDA
Chunk 1 Fold 0.9526515151515151 and method KNN
Chunk 1 Fold 0.9583333333333333 and method QDA
Chunk 1 Fold 0.9147727272727273 and method QDA
Chunk 1 Fold 0.962121212121212 and method SVM
Chunk 1 Fold 0.9772727272727273 and method QDA
Chunk 2 

Chunk 16 Fold 0.9981884057971014 and method QDA
Chunk 16 Fold 0.9764492753623188 and method QDA
Chunk 16 Fold 0.9963768115942029 and method QDA
Chunk 16 Fold 0.9384057971014493 and method QDA
Chunk 16 Fold 0.9545454545454546 and method QDA
Chunk 16 Fold 0.9981060606060607 and method QDA
Chunk 17 Fold 0.9945652173913043 and method QDA
Chunk 17 Fold 0.963768115942029 and method QDA
Chunk 17 Fold 0.9873188405797102 and method QDA
Chunk 17 Fold 0.996376811594203 and method QDA
Chunk 17 Fold 0.9818840579710145 and method QDA
Chunk 17 Fold 0.9692028985507247 and method QDA
Chunk 17 Fold 1.0 and method QDA
Chunk 17 Fold 0.9715909090909091 and method QDA
Chunk 17 Fold 0.9753787878787878 and method QDA
Chunk 17 Fold 0.9486166007905138 and method QDA
Chunk 17 Fold 0.9604743083003953 and method QDA
Chunk 18 Fold 0.9808695652173913 and method QDA
Chunk 18 Fold 0.9764492753623188 and method QDA
Chunk 18 Fold 0.980072463768116 and method QDA
Chunk 18 Fold 1.0 and method QDA
Chunk 18 Fold 0.918478260

Chunk 32 Fold 0.987012987012987 and method QDA
Chunk 32 Fold 0.9761904761904762 and method QDA
Chunk 33 Fold 0.9458333333333333 and method QDA
Chunk 33 Fold 0.9633333333333334 and method QDA
Chunk 33 Fold 0.9666666666666666 and method QDA
Chunk 33 Fold 0.9866666666666667 and method KNN
Chunk 33 Fold 0.9583333333333334 and method QDA
Chunk 33 Fold 0.9756944444444444 and method QDA
Chunk 33 Fold 0.9913194444444444 and method QDA
Chunk 33 Fold 0.8840579710144928 and method QDA
Chunk 33 Fold 0.9963768115942029 and method QDA
Chunk 33 Fold 0.9637681159420289 and method QDA
Chunk 33 Fold 0.9891304347826086 and method QDA
Chunk 34 Fold 0.959866220735786 and method KNN
Chunk 34 Fold 0.9431438127090301 and method QDA
Chunk 34 Fold 0.9686956521739131 and method QDA
Chunk 34 Fold 0.9930434782608696 and method QDA
Chunk 34 Fold 0.9721739130434783 and method QDA
Chunk 34 Fold 0.9618181818181817 and method QDA
Chunk 34 Fold 0.9963636363636365 and method QDA
Chunk 34 Fold 0.9981818181818182 and metho

Chunk 48 Fold 0.9981884057971014 and method QDA
Chunk 49 Fold 0.9655797101449275 and method KNN
Chunk 49 Fold 0.9456521739130436 and method QDA
Chunk 49 Fold 0.9692028985507246 and method QDA
Chunk 49 Fold 0.9556159420289855 and method QDA
Chunk 49 Fold 0.9746376811594203 and method QDA
Chunk 49 Fold 0.9945652173913043 and method QDA
Chunk 49 Fold 0.947463768115942 and method QDA
Chunk 49 Fold 0.9603024574669188 and method QDA
Chunk 49 Fold 0.941398865784499 and method QDA
Chunk 49 Fold 0.9659735349716445 and method QDA
Chunk 49 Fold 1.0 and method QDA
Chunk 50 Fold 0.9892307692307692 and method QDA
Chunk 50 Fold 0.9676923076923077 and method QDA
Chunk 50 Fold 0.9676923076923076 and method QDA
Chunk 50 Fold 0.9615384615384615 and method QDA
Chunk 50 Fold 0.9446153846153846 and method QDA
Chunk 50 Fold 0.96 and method QDA
Chunk 50 Fold 0.9904000000000001 and method QDA
Chunk 50 Fold 1.0 and method QDA
Chunk 50 Fold 0.9666666666666666 and method QDA
Chunk 50 Fold 0.9966666666666667 and m

Chunk 65 Fold 0.9665551839464883 and method QDA
Chunk 65 Fold 0.9916387959866221 and method QDA
Chunk 65 Fold 0.9180602006688963 and method QDA
Chunk 65 Fold 0.98 and method QDA
Chunk 65 Fold 0.9672727272727272 and method QDA
Chunk 65 Fold 0.9727272727272728 and method QDA
Chunk 65 Fold 0.9818181818181818 and method KNN
Chunk 65 Fold 0.990909090909091 and method QDA
Chunk 65 Fold 0.9418181818181819 and method KNN
Chunk 66 Fold 1.0 and method QDA
Chunk 66 Fold 0.9692028985507246 and method KNN
Chunk 66 Fold 0.911231884057971 and method QDA
Chunk 66 Fold 0.9024621212121211 and method QDA
Chunk 66 Fold 0.9980237154150198 and method QDA
Chunk 66 Fold 0.942687747035573 and method QDA
Chunk 66 Fold 0.9723320158102766 and method QDA
Chunk 66 Fold 0.9644268774703557 and method QDA
Chunk 66 Fold 0.9802371541501977 and method QDA
Chunk 66 Fold 0.9901185770750989 and method QDA
Chunk 66 Fold 0.9545454545454546 and method KNN
Chunk 67 Fold 0.9182608695652175 and method KNN
Chunk 67 Fold 0.94260869

...

Chunk 446 Fold 0.8574380165289255 and method KNN
Chunk 446 Fold 0.9524793388429752 and method QDA
Chunk 447 Fold 0.9723320158102767 and method QDA
Chunk 447 Fold 0.9604743083003953 and method QDA
Chunk 447 Fold 0.8873517786561265 and method QDA
Chunk 447 Fold 0.9229249011857708 and method QDA
Chunk 447 Fold 0.9861660079051383 and method QDA
Chunk 447 Fold 0.96900826446281 and method QDA
Chunk 447 Fold 0.9710743801652892 and method QDA
Chunk 447 Fold 0.9917355371900827 and method KNN
Chunk 447 Fold 0.9956709956709957 and method QDA
Chunk 447 Fold 0.9805194805194806 and method QDA
Chunk 447 Fold 0.9805194805194806 and method QDA
Chunk 448 Fold 0.92 and method QDA
Chunk 448 Fold 0.985 and method QDA
Chunk 448 Fold 0.9833333333333333 and method KNN
Chunk 448 Fold 0.9933333333333334 and method QDA
Chunk 448 Fold 0.9766666666666668 and method QDA
Chunk 448 Fold 0.9716666666666667 and method QDA
Chunk 448 Fold 0.9818840579710144 and method QDA
Chunk 448 Fold 0.9655797101449276 and method QDA


Chunk 462 Fold 0.9565217391304348 and method QDA
Chunk 462 Fold 0.9836956521739131 and method QDA
Chunk 463 Fold 0.9773156899810964 and method QDA
Chunk 463 Fold 0.9867674858223063 and method QDA
Chunk 463 Fold 0.9603024574669188 and method QDA
Chunk 463 Fold 0.9697542533081286 and method QDA
Chunk 463 Fold 1.0 and method QDA
Chunk 463 Fold 0.9508506616257089 and method QDA
Chunk 463 Fold 0.9920948616600791 and method KNN
Chunk 463 Fold 0.9723320158102767 and method QDA
Chunk 463 Fold 0.9723320158102766 and method QDA
Chunk 463 Fold 0.9624505928853755 and method QDA
Chunk 463 Fold 0.958498023715415 and method SVM
Chunk 464 Fold 0.980072463768116 and method QDA
Chunk 464 Fold 0.9962192816635161 and method QDA
Chunk 464 Fold 0.9867674858223062 and method QDA
Chunk 464 Fold 0.9943289224952742 and method QDA
Chunk 464 Fold 0.9243856332703213 and method QDA
Chunk 464 Fold 0.9470699432892249 and method QDA
Chunk 464 Fold 0.9678638941398867 and method QDA
Chunk 464 Fold 0.9901185770750988 and

Chunk 478 Fold 0.9861660079051383 and method QDA
Chunk 478 Fold 0.9407114624505929 and method QDA
Chunk 478 Fold 0.950592885375494 and method QDA
Chunk 478 Fold 0.9505928853754941 and method QDA
Chunk 478 Fold 0.9525691699604742 and method QDA
Chunk 479 Fold 1.0 and method QDA
Chunk 479 Fold 0.9666666666666666 and method QDA
Chunk 479 Fold 0.9878260869565217 and method QDA
Chunk 479 Fold 0.977391304347826 and method QDA
Chunk 479 Fold 0.9356521739130435 and method KNN
Chunk 479 Fold 0.9408695652173913 and method QDA
Chunk 479 Fold 0.9652173913043479 and method QDA
Chunk 479 Fold 0.9963768115942029 and method QDA
Chunk 479 Fold 0.9728260869565217 and method QDA
Chunk 479 Fold 0.9927536231884059 and method QDA
Chunk 479 Fold 0.980072463768116 and method QDA
Chunk 480 Fold 0.9340277777777778 and method QDA
Chunk 480 Fold 0.9791666666666666 and method KNN
Chunk 480 Fold 0.9652777777777778 and method KNN
Chunk 480 Fold 0.9166666666666667 and method QDA
Chunk 480 Fold 0.9673913043478262 and 

Chunk 494 Fold 0.9618055555555556 and method QDA
Chunk 494 Fold 1.0 and method QDA
Chunk 494 Fold 0.9930555555555556 and method QDA
Chunk 494 Fold 0.954861111111111 and method QDA
Chunk 494 Fold 0.9965277777777777 and method QDA
Chunk 494 Fold 0.9670138888888888 and method QDA
Chunk 494 Fold 0.9583333333333334 and method QDA
Chunk 495 Fold 0.9716666666666667 and method QDA
Chunk 495 Fold 0.9983333333333333 and method QDA
Chunk 495 Fold 0.9565217391304348 and method KNN
Chunk 495 Fold 0.9982608695652173 and method QDA
Chunk 495 Fold 0.9043478260869565 and method QDA
Chunk 495 Fold 0.9773913043478261 and method QDA
Chunk 495 Fold 0.8913043478260869 and method QDA
Chunk 495 Fold 0.9257246376811594 and method QDA
Chunk 495 Fold 0.9619565217391304 and method QDA
Chunk 495 Fold 0.9184782608695652 and method KNN
Chunk 495 Fold 0.9510869565217391 and method QDA
Chunk 496 Fold 0.931159420289855 and method QDA
Chunk 496 Fold 0.9818840579710144 and method QDA
Chunk 496 Fold 0.9945652173913043 and

Chunk 510 Fold 0.9438405797101449 and method QDA
Chunk 510 Fold 0.9764492753623188 and method QDA
Chunk 510 Fold 0.9402173913043478 and method QDA
Chunk 510 Fold 0.9565217391304347 and method QDA
Chunk 510 Fold 0.8641304347826086 and method KNN
Chunk 510 Fold 0.9673913043478262 and method QDA
Chunk 510 Fold 0.8853754940711462 and method KNN
Chunk 511 Fold 0.9420289855072463 and method QDA
Chunk 511 Fold 1.0 and method QDA
Chunk 511 Fold 0.9924242424242424 and method QDA
Chunk 511 Fold 0.9583333333333334 and method KNN
Chunk 511 Fold 0.9431818181818181 and method QDA
Chunk 511 Fold 0.9703557312252965 and method QDA
Chunk 511 Fold 0.9683794466403162 and method QDA
Chunk 511 Fold 0.9762845849802371 and method QDA
Chunk 511 Fold 0.9881422924901185 and method QDA
Chunk 511 Fold 0.9209486166007904 and method QDA
Chunk 511 Fold 0.8932806324110673 and method QDA

AUC: 0.96308


In [ ]:
oof_temp, preds_temp = instant_model(train1, test1, cols)

In [47]:
newtrain1 = get_newtrain(train1, test1, preds_temp, oof_temp)
cols1 = [c for c in newtrain1.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]


oof, preds = instant_model(newtrain1, test1,cols1)
#oof1, preds1 = instant_model(newtrain1, test1,cols1, clf = NuSVC(probability = True, kernel = 'poly'))
#oof2, preds2 = instant_model(newtrain1, test1,cols1, clf = KNeighborsClassifier(n_neighbors = 7, p = 2, weights = 'distance'))


AUC: 0.99345


In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds
sub.to_csv('submission_stack.csv',index=False)